In [2]:
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Boundary conditions
def apply_bcs(K_gl,F_gl,dof,value):
    phi = 1.0
    bignumber = 10.0**30
    K_gl[dof,dof] += bignumber*phi*phi
    F_gl[dof] += bignumber*float(value)*phi  

In [4]:
# Global index of local DOF
def gl_index(elem_index,local_dof):
    return elem_index + local_dof

In [5]:
# Global stiffness matrix and mass matrix
def GlobalSystem(E,L,A,n_elem,bc1,bc2):
    
    # Element stiffness matrix and element load vector
    h = L/n_elem
    k_elem = E*A/h * np.array([[1.0, -1.0], [-1.0, 1.0]])
    m_elem = rho*A*h/2 * np.array([[1.0,0.0],[0.0,1.0]])
    f_elem = np.array([0.0, 0.0])

    # Global stiffness matrix and load vector
    K_gl = np.zeros((n_elem+1, n_elem+1))
    F_gl = np.zeros((n_elem+1))
    M_gl = np.zeros((n_elem+1, n_elem+1))
    # Assembly of elements
    for i_el in range(n_elem):
        for i_loc in range(2):
            i_gl = gl_index(i_el, i_loc)
            F_gl[i_gl] += f_elem[i_loc]
            for j_loc in range(2):
                j_gl = gl_index(i_el, j_loc)
                K_gl[i_gl,j_gl] += k_elem[i_loc, j_loc]
                M_gl[i_gl,j_gl] += m_elem[i_loc, j_loc]

    # Boundary conditions
    apply_bcs(K_gl, F_gl, 0, bc1)
    # apply_bcs(K_gl, F_gl, n_elem, bc2)

    # Approximate solution
    # uh = np.linalg.solve(K_gl, F_gl)

    return K_gl, M_gl

In [6]:
# Newmark's method implicit
def newmark_imp(K, M, C, u, v, acel, p_next, dt, gamma, beta):
    #acel0 = np.matmul(np.linalg.inv(M),(p0 - C*v0 - K*u0))

    # Degrees of freedom
    dofs = K.shape[0]
    # Integration constants
    alpha = [
        1.0/(beta*dt**2.0),
        gamma/(beta*dt),
        1.0/(beta*dt),
        1.0/(2.0*beta)-1.0,
        gamma/beta-1.0,
        (gamma/beta-2.0)*dt/2.0,
        (1.0-gamma)*dt,
        gamma*dt
        ]

    # Effective stiffness matrix
    Keff = K + alpha[0]*M + alpha[1]*C
    
    # Keff inverse
    Kinv = np.linalg.inv(Keff)
    

    # Integration
    # Vector of effective forces at time
    pef = np.zeros((dofs))
    # Vector of displacements at time
    u_next = np.zeros((dofs))
    # Vector of accelerations  and velocities at time
    acel_next = np.zeros((dofs))
    v_next = np.zeros((dofs))

    # Vector of effective forces p
    term1 = alpha[0]*u + alpha[2]*v + alpha[3]*acel
    term2 = alpha[1]*u + alpha[4]*v + alpha[5]*acel
    pef_next = p_next + np.matmul(M,term1) + np.matmul(C,term2)
    # Vector of displacements u
    u_next = np.matmul(Kinv,pef_next)
    # Vector of accelerations acel and velocities v 
    acel_next = alpha[0]*(u_next-u) - alpha[2]*v - alpha[3]*acel
    v_next = v + alpha[6]*acel + alpha[7]*acel_next  
    
    return u_next,v_next,acel_next


# Newmark's method explicit
def newmark_exp(K, M, C, u, v, acel, p_next, dt, gamma, beta):

    # Degrees of freedom
    dofs = K.shape[0]
    
    up = np.zeros((dofs))
    vp = np.zeros((dofs))
    u_next = np.zeros((dofs))
    acel_next = np.zeros((dofs))
    v_next = np.zeros((dofs))

    # Predictors vectors
    up = u + v*dt + acel*(1.0/2.0 - beta)*dt**2
    vp = v + acel*(1 - gamma)*dt

    # Solution of the linear problem:
    term1 = M + np.multiply(gamma*dt,C) + np.multiply(beta*dt**2,K)
    term1 = np.linalg.inv(term1)
    term2 = p_next - np.multiply(K,up) - np.multiply(C,vp)
    acel_next = np.matmul(term1,term2)

    # Correctors
    u_next = up + acel_next*beta*dt**2
    v_next = vp + acel_next*gamma*dt
    
    return u_next,v_next,acel_next

In [7]:
# Plot functions 

# Plot data in nodes
def plot(x,y,labelx,labely,title):
    fig, axes = plt.subplots()
    axes.grid(True, which='both')
    axes.axhline(y=0, color='k')
    plt.title(str(title))
    plt.xlabel(str(labelx))
    plt.ylabel(str(labely))
    plt.plot(x, y)
    plt.show()

# Plot data in element
def plot_interior(x0,xf,n_elem,func,labelx,labely,title):
    h = float(xf-x0)/n_elem
    x = np.array([[x0+h*el,x0+h*(el+1)] for el in range(n_elem)])
    x = x.flatten()
    y = np.array([[func[el],func[el]] for el in range(n_elem)])
    y = y.flatten()
    plot(x,y,labelx,labely,title)

#Plot data in time
def plot_time(n_steps,func,labelx,labely,title):
    x = np.linspace(0, n_steps, n_steps)
    y = func
    plot(x,y,labelx,labely,title)

# Plot energy 
def plot_energy(n_steps,E_kin,E_pot,labelx,labely,title):
    fig, axes = plt.subplots()
    axes.grid(True, which='both')
    axes.axhline(y=0, color='k')
    plt.title(str(title))
    plt.xlabel(str(labelx))
    plt.ylabel(str(labely))
    x = np.linspace(0, n_steps, n_steps)
    plt.plot(x, E_kin, label='E_kin')
    plt.plot(x, E_pot, label='E_pot')
    plt.show()
    

In [8]:
# Energy balance
def energy(K,M,u,v):
    # Kinetic energy
    E_kin = 1.0/2.0*np.dot(np.matmul(M,v),v)
    # Potential energy
    E_pot = 1.0/2.0*np.dot(np.matmul(K,u),u)
    # Total 
    E_tot = E_kin + E_pot
    return E_kin, E_pot, E_tot

In [9]:
# Global algorithm 

# Input parameters

# Material parameters
E = 275.0*10**3 #(Pa)
L = 0.2 #(m)
A = 0.001 #(unit area)
Gc = 100.0 #(N/m)
stress_c = 300.0*10**3 #(Pa)
rho = 75.0 #(kg/m3)

# Mesh
n_elem = 1
h = L/n_elem
dofs = n_elem + 1

# Time steps
n_steps = 1

# Initial data

# Boundary conditions
strain_rate = 10.0**-4 #(s-1)
# Velocities
bc1 = 1

# Initial displacement (u0)
u0 = np.zeros((dofs))
# Initial velocity (v0): velocity profile (vel) is a function v(x)
vel = strain_rate*L
n_points = dofs
# l = np.linspace(-L/2, L/2, n_points)
l = np.linspace(0, L, n_points)
# v0 = np.array([2*vel/L*x for x in l])
# v0 = np.array([vel/L*x for x in l])
# v0 = np.round(v0, 8)
v0 = np.zeros((dofs))
# v0[0] = -vel
v0[dofs-1] = vel

# Initial acceleration (acel0)
acel0 = np.zeros((dofs))
# Load (p)
p = np.zeros((n_steps+1,dofs))

# Time integration
dt = 10**-4 #(s)
gamma = 0.5
beta = 0.5 
C = np.zeros((dofs,dofs))  

# Get K_gl and M_gl
K,M = GlobalSystem(E,L,A,n_elem,bc1,0)
# print(K)
# print(M)

# Get u,v,a,stain and stress 
u = u0
v = v0
acel = acel0

stress_evolution = np.zeros((n_steps, n_elem))
E_kin = np.zeros((n_steps))
E_pot = np.zeros((n_steps))
E_tot = np.zeros((n_steps))

for n in range(n_steps):

    # plot(l,v,"x","v","Velocity")
    # u,v,acel = newmark_imp(K,M,C,u,v,acel,p[n+1],dt,gamma,beta)
    u,v,acel = newmark_exp(K,M,C,u,v,acel,p[n+1],dt,gamma,beta)
    E_kin[n],E_pot[n],E_tot[n] = energy(K,M,u,v)
    strain = np.zeros(n_elem)
    stress = np.zeros(n_elem)
    for el in range(n_elem):
        strain[el] = (u[el+1] - u[el])/h
        stress[el] = E*strain[el]
        # Matrix with stress in all elements (columns) at all time_steps (lines)
        stress_evolution[n,el] = stress[el]
    
    # Plot of stress in all element (for each time_step a graphic of stress in all elements - according to x coordinate)
    # plot_interior(0,L,n_elem,stress,"x","sigma","stress")

# plot_energy(n_steps,E_kin,E_pot,"time_step","Energy","Energy balance")
print(u)
print(v)
print(acel)


# Plot of the stress evolution on time (for each element a graphic of stress x time_step)
for el in range(n_elem):
    stress_evolution_el = stress_evolution[:,el]
    # plot_time(n_steps,stress_evolution_el,"time_step","stress","stress evolution")